In [2]:
!pip install -qU pip awscli boto3 sagemaker
!pip install nvidia-pyindex
!pip install tritonclient[http]

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
aiobotocore 2.4.0 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.16 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8413 sha256=fd18f46b9edaa43cd9e4019b416b8811be26d3e8c26e8bc984f8cd1d9a3b0498
  Stored in directory: /tmp/pip-ephem-wheel-cache-_9_ksvtd/wheels/65/cd/01/fb75031f3f86f9d8940f46f7c23fc1dcd38965799131e06a7e
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.n

### To create the JIT Trace model you need

Torch version 1.12.1 Torch Vision 0.13.1 and Cuda library 11.3

Secondly you need a GPU instance to run the notebook - this has been tested on a ml.g4dn.xlarge which comes with 1 gpu

In [28]:
pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 171.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 34.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os 
import tarfile
import boto3, json, sagemaker, time
from sagemaker import get_execution_role
import concurrent.futures
import time

### OPTIONAL -- Create a JIT Traced model

#### Few points to note: The traced models is provided in the zip file which can be used as is
1. The Model after trace is now returing outputs like OUTPUT_0 , 1 etc
2. To change them to named outputs can be done and we can try post this issue gets resolved
3. To full create a jit traced model we will need to provide a sample inputs and hence for now we have created a scripted model
4. The TORCH and the TORCHSCRIPT libraries would need to match the container and hence we use the specific ones mentioned above

In [3]:
import torch
from pathlib import Path
print(torch.__version__)
import torchvision
print(torchvision.__version__)


1.12.1+cu113
0.13.1+cu113


In [4]:
torch.cuda.is_available()

True

**Convert the model into Serving mode**

In [6]:
original_model_path = Path("jit-resnet-v3-model/1/model.pt")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(original_model_path)
model = model.to(device)
model = model.eval()

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


In [141]:
%%writefile jit-resnet-v3-model/config.pbtxt
name: "jit-resnet-v3-model"
platform: "pytorch_libtorch"
max_batch_size: 12
input {
  name: "INPUT__0"
  data_type: TYPE_UINT8
  dims: [3,480,856]
}
output {
  name: "OUTPUT__0"
  data_type: TYPE_FP32
  dims: [-1]
}
output {
  name: "OUTPUT__1"
  data_type: TYPE_INT64
  dims: [-1]
}

output {
  name: "OUTPUT__2"
  data_type: TYPE_FP32
  dims: [-1]
}

instance_group {
  count: 3
  kind: KIND_GPU
}

Overwriting jit-resnet-v3-model/config.pbtxt


#### Now create the tar ball and upload 

In [8]:
import requests
from io import BytesIO
from PIL import Image
import concurrent.futures
import tritonclient.http as httpclient
from botocore.config import Config
import numpy as np
import random
import boto3
import time



w,h = 856,480

urls = [
    "https://m.media-amazon.com/images/M/MV5BNDcwZDc2NTEtMzU0Ni00YTQyLWIyYTQtNTI3YjM0MzhmMmI4XkEyXkFqcGdeQXVyNTgyNTA4MjM@._V1_.jpg",
    "https://lh3.googleusercontent.com/05JfZ1ZdyzrRNvhJosUFdcjjJRFE7k2KhmeM2ujqeCbrcrCb1hkq7O_JdUBpQ3r9hi0YeSn4WgmKx3Ai8LHdM2SucxSzl9TRZ4fCAqETJ6WtHgE=s0",
    "https://assets.nintendo.com/image/upload/f_auto/q_auto/dpr_2.625/c_scale,w_400/ncom/en_US/games/switch/n/new-pokemon-snap-switch/hero",
    "https://images.nintendolife.com/d358c9f9118af/pokemon-go.900x.jpg",
    "https://cdn.vox-cdn.com/thumbor/IKt535q8LMnJDddmLL74TBtzv88=/0x266:1024x949/1280x854/cdn.vox-cdn.com/uploads/chorus_image/image/48942277/N3DS_PokemonSuperMysteryDungeon_MainIllustration_png_jpgcopy.0.0.jpg",
    "https://i.imgflip.com/3sn9mp.jpg",
    "https://techcrunch.com/wp-content/uploads/2017/08/cbsn.png"
]



#endpoint_name = m_name
def read_image(i=0):
    url = random.choice(urls)
    
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    img = img.resize((w, h), Image.ANTIALIAS)
    return np.asarray(img, dtype='uint8')



In [91]:
def invoke_endpoint(images,endpoint_name, target_model='model.tar.gz'): # - resnet_fpn_v3.tar.gz
    inputs = []
    outputs = []
    input_data = images # np.asarray(images, dtype='uint8')  # passing in a numpy aray already 
    print(input_data.shape)
    
    #inputs.append(httpclient.InferInput("INPUT__0", [ len(input_data),h, w,3], "UINT8"))
    
    inputs = [httpclient.InferInput("INPUT__0", images.shape, "UINT8")]
    inputs[0].set_data_from_numpy(images, binary_data=True)
    outputs = [httpclient.InferRequestedOutput(f"OUTPUT__{n}", binary_data=True) for n in range(3)]
    
    request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
        inputs, outputs=outputs
    )


    runtime_sm_client = boto3.client(
        "sagemaker-runtime",
        region_name="eu-west-1", 
        config=Config(
            connect_timeout=5,
            read_timeout=60, #120,
            retries={'max_attempts': 2,'mode': 'standard'} #20
        )
    )
    
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/vnd.sagemaker-triton.binary+json;json-header-size={}".format(
            header_length
        ),
        Body=request_body,
        TargetModel=target_model,
        
    )
    header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
    header_length_str = response["ContentType"][len(header_length_prefix) :]
    if not header_length_str:
        header_length_str='0'
    result = httpclient.InferenceServerClient.parse_response_body(response["Body"].read())
    return result


In [9]:
batch_size = 12
samples = 1
buffer = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    images_future = [executor.submit(read_image, i) for i in range(samples*batch_size)]

    for i, future in enumerate(concurrent.futures.as_completed(images_future)):
        buffer.append(future.result())

print(len(buffer))


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.


12


In [10]:
image_batch = np.asarray(buffer, dtype='uint8').transpose(0,3,1,2)  # -- 12 x 3 x 480 x 856 
print(len(buffer[0]))
print(image_batch.shape)

480
(12, 3, 480, 856)


In [21]:
local_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(local_device)
image_array_tensors = torch.tensor(image_batch,device=local_device)
image_array_tensors.shape

cuda


torch.Size([12, 3, 480, 856])

In [22]:
print(len(buffer[0]))
st_time = time.time()
#result = invoke_endpoint(buffer, m_name)
result = model(image_array_tensors)
print(len(buffer),time.time() - st_time)
print(f"Test finished for 1 batch of {batch_size} images::result={result}::")
            

    

480


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1130: UserWarning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (Triggered internally at  ../torch/csrc/jit/codegen/cuda/parser.cpp:3513.)
  return forward_call(*input, **kwargs)


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/torchvision/models/detection/retinanet.py", line 352, in forward
      features = x[_113]
      conv = self.conv
      bbox_regression = (conv).forward(features, )
                         ~~~~~~~~~~~~~ <--- HERE
      bbox_reg = self.bbox_reg
      bbox_regression0 = (bbox_reg).forward(bbox_regression, )
  File "code/__torch__/torch/nn/modules/container/___torch_mangle_42.py", line 19, in forward
    input1 = (_1).forward(input0, )
    input2 = (_2).forward(input1, )
    return (_3).forward(input2, )
            ~~~~~~~~~~~ <--- HERE
  def __len__(self: __torch__.torch.nn.modules.container.___torch_mangle_42.Sequential) -> int:
    return 4
  File "code/__torch__/torchvision/ops/misc/___torch_mangle_41.py", line 13, in forward
    _0 = getattr(self, "0")
    _1 = getattr(self, "1")
    input0 = (_0).forward(input, )
    

In [117]:
result_outputs = result.get_response()['outputs']
for single_output in result_outputs:
    print(f"Each:output:{single_output}")
    print(f"Values:output:{result.as_numpy(single_output['name'])} \n")
    


Each:output:{'name': 'OUTPUT__0', 'datatype': 'FP32', 'shape': [12, 0, 4], 'parameters': {'binary_data_size': 0}}
Values:output:[] 

Each:output:{'name': 'OUTPUT__1', 'datatype': 'INT64', 'shape': [12, 0], 'parameters': {'binary_data_size': 0}}
Values:output:[] 

Each:output:{'name': 'OUTPUT__2', 'datatype': 'FP32', 'shape': [12, 0], 'parameters': {'binary_data_size': 0}}
Values:output:[] 



In [119]:
import numpy as np

print("Starting invocation for model:: please wait ...")
results = []
for i in range(0, 50):
    start = time.time()
    invoke_endpoint(buffer, m_name)
    results.append((time.time() - start) * 1000)
print("\nPredictions for model latency: \n")
print("\nP95: " + str(np.percentile(results, 95)) + " ms\n")
print("P90: " + str(np.percentile(results, 90)) + " ms\n")
print("Average: " + str(np.average(results)) + " ms\n")

Starting invocation for model:: please wait ...

Predictions for model latency: 


P95: 1914.787781238556 ms

P90: 1897.0021724700928 ms

Average: 1874.2761421203613 ms



In [120]:
print("\nPredictions for model latency: \n")
print("\nP95: " + str(np.percentile(results, 95)) + " ms\n")
print("P90: " + str(np.percentile(results, 90)) + " ms\n")
print("Average: " + str(np.average(results)) + " ms\n")


Predictions for model latency: 


P95: 1914.787781238556 ms

P90: 1897.0021724700928 ms

Average: 1874.2761421203613 ms



In [ ]:
    images = read_image()
    inputs = []
    outputs = []
    input_data = np.asarray(images, dtype='uint8')
    inputs.append(httpclient.InferInput("INPUT__0", [ len(input_data),h, w,3], "UINT8"))
    inputs[0].set_data_from_numpy(input_data, binary_data=True)
    outputs.append(httpclient.InferRequestedOutput("BBOX", binary_data=True))
    outputs.append(httpclient.InferRequestedOutput("LABELS", binary_data=True))
    outputs.append(httpclient.InferRequestedOutput("SCORES", binary_data=True))
    request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
        inputs, outputs=outputs
    )


    runtime_sm_client = boto3.client("sagemaker-runtime",region_name="eu-west-1", config=Config(connect_timeout=5,
                                                                                 read_timeout=120,
                                                                                 retries={
                                                                                     'max_attempts': 20,
                                                                                     'mode': 'standard'

                                                                                 }))
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/vnd.sagemaker-triton.binary+json;json-header-size={}".format(
            header_length
        ),
        Body=request_body,
        
    )
    header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
    header_length_str = response["ContentType"][len(header_length_prefix) :]
    result = httpclient.InferenceServerClient.parse_response_body(
        response["Body"].read(), 
        header_length=int(header_length_str)
    )
    return result


In [ ]:
print('hello')